# LIBRARY

In [ ]:
!pip install https://github.com/scikit-learn-contrib/scikit-learn-extra/archive/master.zip

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

from sklearn.metrics import silhouette_score

from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids

import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestClassifier, StackingClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC, SVC, NuSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression, RidgeClassifier, RidgeClassifierCV
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB, CategoricalNB
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_selection import mutual_info_classif
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import StratifiedKFold, KFold, cross_val_score, RepeatedStratifiedKFold
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, make_scorer, recall_score, auc, roc_auc_score
from sklearn.metrics import f1_score, balanced_accuracy_score, fbeta_score, precision_recall_curve, roc_curve
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler
from category_encoders import LeaveOneOutEncoder, MEstimateEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, FunctionTransformer, OrdinalEncoder, OneHotEncoder

# DATA

In [ ]:
path = "/kaggle/input/data-laprak-datmin-1/data indo 2021.xlsx"
data0 = pd.read_excel(path)

Data yang digunakan dalam analisis mencakup variabel-variabel sebagai berikut :
1. Provinsi : Letak provinsi dari Kabupaten/Kota
2. Kab/Kota : Kabupaten/Kota yang digunakan dan akan dianalisis datanya
3. Penduduk_Miskin : Presentase penduduk miskin (P0) menurut Kabupaten/Kota (persen)
4. Lama_Sekolah : Rata-rata lama sekolah penduduk berusia 15 tahun keatas
5. Pengeluaran : Pengeluaran per kapita disesuaikan (ribu rupiah/orang/tahun)
6. IPM : Indeks Pembangunan Manusia
7. UHH : Umur Harapan Hidup (Tahun)
8. Akses_Sanitasi : Persentase rumah tangga yang memiliki akses terhadap sanitasi layak
9. Akses_Minum : Persentase rumah tangga yang memiliki akses terhadap air minum layak
10. TPT : Tingkat Pengangguran Terbuka
11. TPAK : Tingkat partisipasi Angkatan Kerja
12. PDRB : PDRB atas Dasar Harga Konstan menurut Pengeluaran (Rupiah)

# PREPROCESSING (Part 1)

> # Pengecekan Tipe Data & Ukuran Data

In [ ]:
display(data0, data0.info())

Terdapat 12 variabel (columns) dengan 514 observasi (rows)

> # Ubah Nama Kolom
Bertujuan dalam mempermudah analisis

In [ ]:
data0.columns = ["Provinsi", "Kab/Kota", "Penduduk_Miskin", "Lama_Sekolah", "Pengeluaran", "IPM", "UHH", "Akses_Sanitasi", "Akses_Minum", "TPT", "TPAK", "PDRB"]
data0

> # Pengecekan Missing Value

In [ ]:
msno.matrix(data0)

In [ ]:
missing_data = pd.DataFrame({'total_missing_value': data0.isnull().sum()})
missing_data

Berdasarkan tabel total missing value di atas, didapati bahwa pada seluruh kolom atau seluruh variabel tidak terdapat missing value, sehingga data dapat digunakan untuk tahapan analisis lebih lanjut

> # Pengecekan Data Duplikat

In [ ]:
data0.duplicated().sum()

Tidak terdapat data (rows) terduplikat

# EXPLORATORY DATA ANALYSIS (EDA) Part 1

> #### Define numerical and caterical columns

In [ ]:
#Numeric
numeric = data0.loc[:, (data0.dtypes == int) | (data0.dtypes == float)].columns.tolist()
numeric

In [ ]:
#Categoric
categoric = data0.loc[:, (data0.dtypes == object)].columns.tolist()
categoric

> # Statistika Deskriptif

> ### Numerik

In [ ]:
data0.describe()

> ### Kategorik

In [ ]:
data0.select_dtypes("object").describe()

Jawa Timur menjadi provinsi dengan jumlah Kab/Kota terbanyak, yaitu 38 Kab/Kota

> # Plot Distribusi Variabel Numerik

> ### Density Plot

In [ ]:
plt.figure(figsize=(8, 10))
for i in range(0, len(numeric)):
    plt.subplot(4, 3,i+1)
    sns.distplot(data0[numeric[i]], color='orange')
    plt.tight_layout()

**Interpretasi**\
Histogram dari variabel numerik di atas dapat dijadikan acuan dalam menganalisis distribusi dari indikator-indikator TPT dan TPAK, dijelaskan analisis setiap variabel sebagai berikut :
1. Histogram pada variabel presentase penduduk miskin menunjukkan kemencengan positif, dimana berdasarkan plot dapat diartikan bahwa mayoritas Kabupaten/Kota di Indonesia memiliki presentase penduduk miskin yang sangat kecil, pada histogram presentase penduduk miskin ditunjukkan bahwa sebagian besar presentase bernilai dibawah 20%. 
2. Histogram variabel rata-rata lama sekolah menujukkan kemencengan negatif,selain itu ditunjuukan bahwa jumlah frekuensi rata-rata lama sekolah tertinggi berada dibawah angka 10, mengindikasikan bahwa sebagian besar Kabupaten/Kota di Indonesia memiliki rata-rata lama sekolah kurang dari 10 tahun dimana setara dengan pendidikan terakhir SMP, sedangkan hanya sebagian kecil sisanya yang memiliki rata-rata lama sekolah di atas 10 tahun.
3. Pada variabel pengeluaran perkapita histogram hampir mendekati distribusi normal dengan sedikit skewnes ke kanan, Hal tersebut menunjukkan bahwa pengeluaran perkapita tiap kabupaten/kota cukup terdistribusi secara merata dengan rata-rata pengeluaran per kapita kurang lebih sebesar 10000 (dalam ribu rupiah/per orang/tahun).
4. Histogram variabel IPM menujukkan kemencengan negatif, dimana mayoritas nilai IPM tiap kabupaten/kota berada di atas 50 dengan beberapa diantaranya melebihi nilai 75. Hal tersebut mengindikasikan bahwa sebagian besar Kabupaten/Kota di Indonesia telah mencapai IPM pada level sedang hingga tinggi.
5. Pada Variabel Rata-rata usia harapan hidup, histogram menujukkan kemencengan negatif. Pada plot didapati bahwa hampir seluruh nilai rata-rata usia harapan hidup tiap Kabupaten/Kota di Indonesia berapa pada interval 60-80 tahun.
6. Histogram dari variabel akses sanitasi menujukkan kemiringan negatif, dimana pada plot frekuensi nilai tertinggi berada pada interval 50-100. Hal tersebut menujukkan bahwa presentasi rumah tangga yang memiliki akses terhadap sanitasi layak per Kabupaten/Kota sebagian besar telah mencapai angka di atas 50%, bahkan tidak sedikit yang mencapai angka 100%.
7. Histogram dari variabel akses minum menujukkan kemiringan negatif, dimana pada plot frekuensi nilai tertinggi berada pada interval 50-100. Hal tersebut menujukkan bahwa presentasi rumah tangga yang memiliki akses terhadap sanitasi layak per Kabupaten/Kota sebagian besar telah mencapai angka di atas 50%, bahkan frekuensi tertinggi nilai presentase berada pada angka 100%.
8. Pada variabel TPT didapati histogram sedikit menceng ke kiri atau skewness positif, dimana frekuensi nilai mendoinasi pada interval 0-10. Hal tersebut berarti bahwa Tingkat Pengangguran Terbuka pada sebagian besar Kabupaten/Kota di Indonesia berada pada angka 0%-10%.
9. Pada variabel TPAK didapati histogram sedikit menceng ke kiri atau skewness positif, dimana frekuensi nilai mendoinasi pada interval 60-80. Hal tersebut berarti bahwa Tingkat Partisipasi angkatan kerja pada sebagian besar Kabupaten/Kota di Indonesia berada pada angka 60%-80%.
10. Histogram variabel PDRB menunjukkan skewness positif, dimana didapati bahwa seluruh nilai PDRB tiap Kabupaten/Kota berada di atas angka 0.

> ### Boxplot

In [ ]:
plt.figure(figsize=(12, 4))
for i in range(0, len(numeric)):
    plt.subplot(2, 5,i+1)
    sns.boxplot(x=data0[numeric[i]], color='#ff8a1e')
    plt.tight_layout()

**Interpretasi**\
Pada interpretasi plot histogram, analisis eksplorasi lebih menyoroti pada distribusi data. Pada interpretasi box plot ini, analisis eksplorasi akan melihat bagaimana pola outlier pada setiap variabel. Adapun analisis box plot diinterpretasikan sebagai berikut :
* (Outlier Bawah). Variabel-variabel dengan nilai outlier bawah yaitu Usia Harapan Hidup, Akses sanitasi layak, serta akses minum layak. Adanya outlier bawah mengindikasikan terdapat ketimpangan yang negatif pada beberapa Kabupaten/Kota di Indonesia. Hal tersebut berarti bahwa terdapat beberapa Kabupaten/Kota yang tertinggal dari Kabupaten/Kota lain pada bidang usia harapan hidup, akses sanitasi, serta akses minum layak.
* (Outlier Atas). Terdapat outlier atas pada boxplot mengindikasikan terdapat nilai yang jauh lebih tinggi daripada mayoritas nilai lainnya. Variabel dengan outlier atas adalah presentase penduduk miskin, pengeluaran perkapita, TPT,TPAK, dan PDRB. Adanya outlier atas tidak selalu berarti positif untuk seluruh variabel. Pada variabel presentase penduduk miskin serta TPT justru mengindikasikan ketimpangan yang negatif, dimana nilai yang terlalu tinggi dapat diartikan bahwa terdapat Kab./Kota dengan presentase kemiskinan dan TPT yang cukup tinggi. Sedangkan pada variabel pengeluaran dan TPAK nilai outlier atas menjadi sinyal yang bagus, dimana menandakan bahwa terdapat beberapa Kab./Kota dengan TPAK serta PDRB tinggi dan ekonomi yang berkembang sehingga memiliki nilai pengeluaran per kapita yang tinggi.
* (Outlier Atas dan Bawah). Variabel yang terdapat nilai oiutlier atas dan bawah adalah IPM dan rata-rata lama sekolah. Pada kedua variabel ini, adanya outlier bawah mengindikasikan masih terdapat Kab/Kota yang tertinggal secara pendidikan dan IPM dibandingkan dengan sebagian besar Kab./Kota. Sedangkan outlier atas pada variabel tersebut mengindikasikan terdapat Kab/Kota yang lebih unggul pada bisang pendidikan dan level IPM.

> # Plot Korelasi Antar Variabel

In [ ]:
plt.figure(figsize=(10, 8))
mask=np.triu(np.ones_like(data0[numeric].corr(), dtype=bool))
correlation_plot=sns.heatmap(data0[numeric].corr(), annot=True, vmax=1, vmin=-1, center=0, cmap='inferno', mask=mask)

**Interpretasi**\
Nilai Korelasi yang akan dibahas secara tuntas adalah nilai korelasi antara variabel TPT dan TPAK terhadap variabel-variabel lain. Pada variabel TPT nilai korelasi tertinggi atau hubungan tererat didapati pada korelasi antara variabel TPT dengan IPM dan TPAK dimana secara mutlak nilai korelasinya sebesar 0.58. Variabel IPM memberikan pengaruh secara positif, yaitu ketika terjadi peningkatan pada variabel IPM maka akan disertai peningkatan pada variabel TPT. Sedangakn variabel TPAK berhubungan secara linearitas negatif, yang berkebalikan dengan variabel IPM. Adapun korelasi terendah variabel TPT didapati pada variabel Akses Sanitasi dengan korelasi sebesar 0.31 dan pola hubungan linearitas positif.\
Kemudian untuk variabel TPAK, 2 variabel yang paling berpengaruh terhadap nilai TPAK adalah variabel TPT dan IPM dengan nilai korelasi berturut-turt sebesar 0.58 dan 0.51, dimana pola hubungan keduanya adalah linearitas negatif, yang berarti bahwa pningkatan pada variabel TPT dan IPM akan mengakibatkan penurunan nilai pada variabel TPAK. Sedangkan korelasi terendah didapati antara variabel TPAK dengan variabel PDRB sebesar 0.21 dengan pola hubungan linaritas negatif.

> # Bar Plot TPT 10 Kabupaten/Kota Tertinggi

In [ ]:
#membuat data frame antara variabel kabupaten/kota dengan TPT, untuk diambil 10 nilai TPT tertinggi
tpt_kab=pd.DataFrame(data0[["Kab/Kota","TPT"]])
tpt_kab["TPT"]=round(tpt_kab["TPT"],4)
tpt_kab10=tpt_kab.sort_values(by="TPT", ascending=False).head(10)
tpt_kab10

In [ ]:
fig,ax=plt.subplots(figsize=(8,5))
ax=sns.barplot(palette="inferno",ax=ax, x="TPT", y="Kab/Kota", data=tpt_kab10)
plt.title('10 Kabupaten/Kota dengan Tingkat Pengangguran Terbuka Tertinggi', fontsize=12)

        
rects = ax.patches
# Place a label for each bar
for rect in rects:
    # Get X and Y placement of label from rect
    x_value = rect.get_width()
    y_value = rect.get_y() + rect.get_height() / 2

    # Number of points between bar and label; change to your liking
    space = -59
    # Vertical alignment for positive values
    ha = 'left'

    # If value of bar is negative: place label to the left of the bar
    if x_value < 0:
        # Invert space to place label to the left
        space *= -1
        # Horizontally align label to the right
        ha = 'right'

    # Use X value as label and format number
    label = '{:}'.format(x_value)

    # Create annotation
    plt.annotate(
        label,                      # Use `label` as label
        (x_value, y_value),         # Place label at bar end
        xytext=(space, 0),          # Horizontally shift label by `space`
        textcoords='offset points', # Interpret `xytext` as offset in points
        va='center',                # Vertically center label
        ha=ha,                      # Horizontally align label differently for positive and negative values
        color = 'white')            # Change label color to white

**Interpretasi**<br>
Tingkat Pengangguran Terbuka (TPT) merupakan presentase jumlah pengangguran terhadap angkatan kerja. Semakin tinggi nilai TPT pada suatu daerah dapat mengindikasikan bahwa banyak terdapat penduduk angkatan kerja yang tidak terserap dengan baik pada dunia kerja. Berdasarkan Plot 10 Kabupaten/Kota di Indonesia dengan nilai TPT tertinggi, didapati bahwa Kabupaten/Kota dengan nilai TPT tertinggi adalah Kota Padang dengan TPT sebesar 13.3731% yang artinya per 100 orang angkatan kerja (yaitu penduduk berusia 15 tahun ke atas yang bekerja, atau punya pekerjaan namun sementara tidak bekerja dan pengangguran.) terdapat 13 orang yang menganggur. Kemudian nilai TPT tertinggi kedua terdapat pada Kota Makassar dengan TPT sebesar 13.176%, dan disusul dengan Kota Cimahi dengan nilai TPT sebesar 13.0685%.

> # Bar Plot TPAK 10 Kabupaten/Kota Tertinggi

In [ ]:
tpak_kab=pd.DataFrame(data0[["Kab/Kota","TPAK"]])
tpak_kab["TPAK"]=round(tpak_kab["TPAK"],4)
tpak_kab10=tpak_kab.sort_values(by="TPAK", ascending=False).head(10)
tpak_kab10

In [ ]:
fig,ax=plt.subplots(figsize=(8,5))
ax=sns.barplot(palette="inferno",ax=ax, x="TPAK", y="Kab/Kota", data=tpak_kab10)
plt.title('10 Kabupaten/Kota dengan Tingkat Partisipasi Angkatan Kerja Tertinggi', fontsize=12)
        
rects = ax.patches
# Place a label for each bar
for rect in rects:
    # Get X and Y placement of label from rect
    x_value = rect.get_width()
    y_value = rect.get_y() + rect.get_height() / 2

    # Number of points between bar and label; change to your liking
    space = -59
    # Vertical alignment for positive values
    ha = 'left'

    # If value of bar is negative: place label to the left of the bar
    if x_value < 0:
        # Invert space to place label to the left
        space *= -1
        # Horizontally align label to the right
        ha = 'right'

    # Use X value as label and format number
    label = '{:}'.format(x_value)

    # Create annotation
    plt.annotate(
        label,                      # Use `label` as label
        (x_value, y_value),         # Place label at bar end
        xytext=(space, 0),          # Horizontally shift label by `space`
        textcoords='offset points', # Interpret `xytext` as offset in points
        va='center',                # Vertically center label
        ha=ha,                      # Horizontally align label differently for positive and negative values
        color = 'white')            # Change label color to white

**Interpretasi**\
Tingkat Partisipasi Angkatan Kerja (TPAK) adalah persentase banyaknya angkatan kerja terhadap banyaknya penduduk yang berumur 15 tahun ke atas atau penduduk usia kerja. Tingginya angka TPAK pada suatu daerah merupakan indikasi yang positif, dimana hal tersebut menandakan bahwa terdapat banyak penduduk usia kerja yang siap untuk terjun dalam dunia kerja dan memberikan kontribusi positif dalam pembangunan ekonomi. Kabupaten/Kota di Indonesia dengan nilai TPAK tertinggi adalah Kabupaten Nduga dengan nilai TPAk sebesar 97.9339%. Angka tersebut berarti bahwa per 100 penduduk usia kerja di Kabupaten Nduga terdapat 98 penduduk yang merupakan angkatan kerja. Hal tersebut juga berarti bahwa hanya terdapat 2 penduduk usia kerja yang bukan merupakan angkatan kerja. Selanjutnya pada posisi kedua terdapat Kabupaten Mamberamo Tengah dengan nilai TPAK 95.8363% serta posisi ketiga terdapat Kabupaten Lanny Jaya dengan nilai TPAK sebesar 95.6585%. Jika diperhatikan hampir seluruh Kabupaten/Kota dengan nilai TPAK tertinggi berada pada Provinsi Papua dan Papua Barat. hal tersebut dapat terjadi karena faktor jumlah penduduk, rasio penduduk usia kerja, dll.

# CLUSTERING
Digunakan metode **K-MEANS**, untuk mengklaster Tantangan Kerja di Indonesia Per-Kab/Kota Berdasarkan Tingkat Partisipasi Angkatan Kerja (TPAK) & Tingkat Pengangguran Terbuka (TPT)

> # Penentuan K Optimal

> ### Menggunakan Silhouette Scores

In [ ]:
wcss = []
silhouette_scores = []
k_values = range(2, 11)

klast_data = data0[["TPAK", "TPT"]]

for k in k_values:
    kmeans = KMeans(n_clusters=k, n_init='auto')
    kmeans.fit(klast_data)
    wcss.append(kmeans.inertia_)
    labels = kmeans.predict(klast_data)
    silhouette_scores.append(silhouette_score(klast_data, labels))

best_k = k_values[np.argmax(silhouette_scores)]

# Print the results
print("---------------------------------------")
print("Silhouette scores:", silhouette_scores)
print("Best K:", best_k)

In [ ]:
s_scores = pd.DataFrame({"K": k_values, "Silhouette Scores": silhouette_scores})
s_scores.sort_values("Silhouette Scores", ascending = False)

**Interpretasi**\
Berdasarkan kandidat K = 2, 3, ..., 11, diperoleh Shilouette Scores tertinggi sebesar 0.4636 untuk K = 3. Oleh karena itu, akan dibentuk 3 klaster tantangan kerja

> ### Menggunakan Grafik Elbow

In [ ]:
wcss = []

# Specify the range of cluster numbers to try
num_clusters_range = range(1, 11)

# Perform K-Means clustering for different numbers of clusters
for num_clusters in num_clusters_range:
    kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init='auto')
    kmeans.fit(klast_data)
    wcss.append(kmeans.inertia_)

# Plot the WCSS values against the number of clusters
plt.plot(num_clusters_range, wcss, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.title('Elbow Method')
plt.show()

**Interpretasi**\
Berdasarkan grafik di atas, dapat dilihat bahwa nilai wcss (Within-Cluster Sum of Squares) menurun secara signifikan saat jumlah klaster meningkat dari 1 hingga 3. Namun, setelah mencapai 3 klaster, penurunan wcss tidak lagi signifikan dan mulai melambat. Pada titik ini, terdapat elbow yang terlihat pada grafik. Kesimpulan yang sama dengan menggunakan shiloette scores, akan dibentuk 3 klaster tantangan kerja

> # Pembentukan 3 Klaster Tantangan Kerja 

In [ ]:
kmeans = KMeans(n_clusters = 3, random_state=42)

# Fit the K-Means model to the data
kmeans.fit(klast_data)

# Obtain the cluster labels for each data point
labels = kmeans.labels_

centroids = kmeans.cluster_centers_

In [ ]:
klast_data["Klaster"] = labels
klast_data

> ### Formatting Data (Urutkan Label Klaster)

In [ ]:
klast_data.groupby('Klaster').agg(['mean','median'])

In [ ]:
klast_data['Klaster'] = klast_data['Klaster'].replace({0: 2, 1: 1, 2: 3})
labels = klast_data["Klaster"]

> # Karakterisasi Tiap Klaster

In [ ]:
klast_data.groupby('Klaster').agg(['mean','median'])

**Interpretasi**\
Terbentuk 3 Klaster dengan
* Klaster 1: Tantangan Kerja Tinggi, dengan Tingkat Partisipasi Angkatan Kerja (TPAK) terendah dan Tingkat Pengangguran Terbuka (TPT) Tertinggi
* Klaster 2: Tantangan Kerja Sedang, dengan Tingkat Partisipasi Angkatan Kerja (TPAK) menengah dan Tingkat Pengangguran Terbuka (TPT) menengah
* Klaster 3: Tantangan Kerja Rendah, dengan Tingkat Partisipasi Angkatan Kerja (TPAK) tertinggi dan Tingkat Pengangguran Terbuka (TPT) terendah

> ### Jumlah Kab/Kota Tiap Klaster

In [ ]:
klast_data.Klaster.value_counts()

**Interpretasi**<br>
* Terdapat 226 Kab/Kota yang masuk klaster 1 (Tantangan Kerja Tinggi)
* Terdapat 246 Kab/Kota yang masuk klaster 2 (Tantangan Kerja Sedang)
* Terdapat 42 Kab/Kota yang masuk klaster 3 (Tantangan Kerja Rendah)

> ### Contoh Kab/Kota Tiap Klaster

In [ ]:
print("Klaster 1 \n", data0["Kab/Kota"][klast_data.Klaster == 1].sample(n = 3, random_state=42), "\n")
print("Klaster 2 \n", data0["Kab/Kota"][klast_data.Klaster == 2].sample(n = 3, random_state=42), "\n")
print("Klaster 3 \n", data0["Kab/Kota"][klast_data.Klaster == 3].sample(n = 3, random_state=42))

# EDA (Part 2)

In [ ]:
data_klaster = data0.copy()
data_klaster["Klaster"] = labels
data_klaster

> # Sebaran Klaster
Berdasarkan TPAK dan TPT

In [ ]:
# Membuat scatter plot untuk setiap nilai unik dalam labels
for label in np.unique(labels):
    plt.scatter(klast_data['TPAK'][labels == label], klast_data['TPT'][labels == label], cmap='viridis', label=f'Cluster {label}')

# Menampilkan centroid
plt.scatter(centroids[:, 0], centroids[:, 1], marker='x', color='red', s=100, label='Centroids')

# Menambahkan keterangan dan judul
plt.xlabel('TPAK')
plt.ylabel('TPT')
plt.title('K-Means Clustering Results')

# Menampilkan legend
plt.legend()

# Menampilkan plot
plt.show()

> ### Centroidnya Tiap Klaster

In [ ]:
centroids # Klaster 2, 1, 3

> # Plot Distribusi Data Setiap Kluster Berdasarkan TPT

In [ ]:
sns.displot(data_klaster, x="TPT", hue="Klaster", kde=True, palette="viridis")

**Interpretasi**\
Plot histogram dari Nilai TPAK berdasarkan kategori tiap klaster ditujukan untuk dapat melihat perbandingan pesebaran atau distribusi nilai TPT antar klaster. Pada klaster 1 yang ditunjukkan dengan histogram berwarna ungu, dapat dilihat bahwa distribusi data hampir mendekati distribusi normal dengan frekuensi nilai TPT berada pada interval 2-14. Kemudian ada klaster 2 yang ditandai dengan histogram berwarna hujau, dapat dilihat bahwa distribusi data cenderung skewness positif, dimana frekuensi tertinggi nilai TPT berada pada range 2 s.d. 5. Kemudian pada klaster kedua yang ditunjukkan oleh histogram berwarna kuning, dilihat bahwa frekuensi TPT mendominasi pada interval 0-4. TPT merupakan variabel yang diharapkan dapat diminimumkan nilainya, dikarenakan tingginya nilai TPT mengindikasikan banyaknya angkatan kerja yang tidak terserap dalam dunia kerja. Berdasarkan ditribusi niai TPT tiap klaster dapat disimpulkan bahwa urutan klaster dengan TPT tertinggi hingga terendah adalah klaster 1, klaster 2, kemudian klaster 3.

> # Plot Distribusi Data Setiap Kluster Berdasarkan TPAK

In [ ]:
sns.displot(data_klaster, x="TPAK", hue="Klaster", kde=True, palette="viridis")

**Interpretasi**\
Histogram dari nilai TPAK berdasarkan tiap klaster ditujukan untuk melihat perbandingan pesebaran atau distribusi dari nilai TPAK antar klaster. Pada klaster 1 dengan histogram berwarna ungu, dapat dilihat bahwa histogram membentuk skewness positif dengan sebaran nilai TPAk berada pada sekitar nilai minimum hingga angka 70. Kemudian pada klaster 2 yang ditandai dengan histogram berwarna hijau, dapat dilihat bahwa histogam mendekati distribusi normal dengan interval nilai TPAK berada di sekitar 65 s.d. 80. Kemudian pada klaster 3 yang ditunjukkan dengan histogram berwarna kuning dapat dilihat bahwa histogram membentuk skewness negatif,dengan sebaran nilai TPAk berada pada interval 80 s.d 100. Variabel TPAK merupakan variabel yang diharapkan dapat dimaksimumkan nilainya, dikarenakan nilai TPAK yang tinggi menjadi indikasi bahwa sebagian besar penduduk usia kerja merupakan angkatan kerja. Berdasarkan nilai TPAK diperoleh urutan klaster dengan nilai TPAK dari yang terendah ke tertinggi adalah Klaster 1, Klaster 2, dan terakhir Klaster 3.

> # Distribusi Indikator TPT dan TPAK Berdasarkan Klaster

In [ ]:
#Numeric
numeric_klas = data_klaster.loc[:, (data_klaster.dtypes == int) | (data_klaster.dtypes == float)].columns.tolist()
numeric_klas

In [ ]:
plt.figure(figsize=(15, 10))
for i in range(0, len(numeric)):
    plt.subplot(3,4,i+1)
    sns.boxplot(x=data_klaster["Klaster"], y=data_klaster[numeric[i]], palette='inferno')
    plt.tight_layout()

In [ ]:
sns.set_style('darkgrid')
sns.histplot(data = data_klaster, x='Klaster', kde=False, bins=5, color='skyblue')
plt.title('Distribusi Klaster')
plt.xlabel('Klaster')
plt.ylabel('Jumlah Kab/Kota')
plt.xticks([1.18, 2, 2.8], ['1', '2', '3'])
plt.show()

**Interpretasi**\
Berdasarkan plot di atas, dapat dilihat bahwa masih sedikit Kab/Kota di Indonesia yang masuk kedalam klaster tantangan kerja rendah, serta lebih di dominasi pada klaster tantangan kerja sedang dan tinggi. Artinya terdapat indikasi kondisi tidak seimbang pada data (imbalanced data)

### Kesimpulan EDA Part 2

Berdasarkan hasil analisis eksplorasi dengan melihat perbandingan pesebaran atau distribusi nilai TPT dan TPAK antar klaster, dapat diperoleh urutan klaster dari tingkatan rendah ke paling tinggi. Suatu klaster dikategorikan tingkatan paling tinggi apabila memiliki interval sebaran nilai TPT terendah sedangkan sebaran nilai TPAK menduduki interval tertinggi. Klaster tersebut mengindikasikan tantangan tenaga kerja yang rendah. Sedangkan nilai sebaliknya, dapat menjadi indikasi bahwa pada klaster tersebut didapati tantangan kerja yang tinggi. Berdasarkan hal tersebut, maka klaster 1 merupakan klaster dengan tantangan tenaga kerja tertinggi, disusul dengan klaster 2, dan klaster dengan tingkat tantangan tenaga kerja terendah adalah klaster 3. Perlu adanya tindakan terhadap klaster dengan tantangan tenaga kerja yang tinggi dengan dilakukan analisis lebih lanjut.

# KLASIFIKASI

In [ ]:
data= data0.drop(["TPAK", "TPT"], axis = 1)
data["Klaster"] = labels
data

In [ ]:
data.Klaster.value_counts()

> # Splitting Data
70% training data, 30% testing data, gunakan stratify karena imbalanced data

In [ ]:
X = data.drop(["Kab/Kota", "Klaster", "Provinsi"], axis = 1)
y = data.Klaster

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_uji, y_train, y_uji = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)
display(X_train, X_uji)

In [ ]:
# X_train.describe()
y_train.value_counts()

# Modelling

> # Preprocessing (Part 2) Data Menggunakan Pipeline
Dilakukan scaling pada feature numerik dengan metode ... (masih milih) menggunakan pipeline

In [ ]:
X1 = X_train.copy()
y1 = y_train.copy()

# kat-num columns
kat_kol = X1.select_dtypes("object").columns
num_kol = X1.select_dtypes(exclude="object").columns

# Pipeline
num_trf1 = Pipeline(steps=[("scaler", MinMaxScaler())])

num_trf2 = Pipeline(steps=[("scaler", RobustScaler())])

num_trf3 = Pipeline(steps=[("scaler", StandardScaler())])

pre_1 = ColumnTransformer(transformers=[
    ("num", num_trf1, num_kol)
])

pre_2 = ColumnTransformer(transformers=[
    ("num", num_trf2, num_kol)
])

pre_3 = ColumnTransformer(transformers=[
    ("num", num_trf3, num_kol)
])

# Modelling
pipa_CB = imbpipeline(steps=[
    ("preprocess", pre_1),
#     ("over", RandomOverSampler(random_state=42)),
    ("clf",  CatBoostClassifier(random_state=0))
])

pipa_RF1 = imbpipeline(steps=[
    ("preprocess", pre_1),
#     ("over", RandomOverSampler(random_state=42)),
    ("clf", RandomForestClassifier(random_state=42))
])

pipa_RF2 = imbpipeline(steps=[
    ("preprocess", pre_2),
    ("clf", RandomForestClassifier(random_state=42))
])

pipa_RF3 = imbpipeline(steps=[
    ("preprocess", pre_3),
    ("clf", RandomForestClassifier(random_state=42))
])

pipa_XG1 = imbpipeline(steps=[
    ("preprocess", pre_1),
#     ("over", RandomOverSampler(random_state=42)),
    ("clf", XGBClassifier(n_estimators = 223,
                          random_state=0))
])

> # Baseline Model

In [ ]:
pip install lazypredict

In [ ]:
from lazypredict.Supervised import LazyClassifier

In [ ]:
X_train1 = pd.DataFrame(pre_1.fit_transform(X1))
X_test1 = pd.DataFrame(pre_1.transform(X_uji))
X_train2 = pd.DataFrame(pre_2.fit_transform(X1))
X_test2 = pd.DataFrame(pre_2.transform(X_uji))
X_train3 = pd.DataFrame(pre_3.fit_transform(X1))
X_test3 = pd.DataFrame(pre_3.transform(X_uji))

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def f1_micro(y_true, y_pred):
    # Menghitung presisi (precision) dan recall secara keseluruhan
    precision = precision_score(y_true, y_pred, average='micro')
    recall = recall_score(y_true, y_pred, average='micro')
    
    # Menghitung F1-score menggunakan presisi dan recall
    f1_mic = 2 * (precision * recall) / (precision + recall)
    
    return f1_mic

# Menggunakan custom F1-score micro dalam LazyClassifier
lazy_clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=f1_micro)

In [ ]:
# lazy_clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models1,predictions1 = lazy_clf.fit(X_train1, X_test1, y1, y_uji)
models2,predictions2 = lazy_clf.fit(X_train2, X_test2, y1, y_uji)
models3,predictions3 = lazy_clf.fit(X_train3, X_test3, y1, y_uji)

In [ ]:
models1

In [ ]:
table= pd.DataFrame()
table["prepo1"] = models1["f1_micro"].sort_values(ascending=False)
table["prepo2"] = models2["f1_micro"].sort_values(ascending=False)
table["prepo3"] = models3["f1_micro"].sort_values(ascending=False)
table

sama aja wkwkwkk

## Hyperparemeter Tuning

> ### LGBM

In [ ]:
#LGBM
pipa_LGBM = imbpipeline(steps=[
    ("preprocess", pre_3),
    #("over", RandomOverSampler(random_state=42)),
    ("clf", LGBMClassifier(random_state=56))
])

# Daftar parameter yang ingin dituning
param_grid = {
    'clf__n_estimators': [150],
    'clf__max_depth': [16],
    'clf__max_bin': [120],
    'clf__learning_rate': [0.2],
    'clf__num_iterations': [150]
}

scorer = make_scorer(f1_score, average='micro')

# Inisialisasi GridSearchCV
grid_search = GridSearchCV(estimator=pipa_LGBM, param_grid=param_grid, cv=5, verbose = 2, scoring=scorer)

# Melakukan pencarian parameter terbaik
grid_search.fit(X1, y1)

# Menampilkan parameter terbaik
print("Best Parameters:", grid_search.best_params_)
# Melakukan prediksi dengan model terbaik
best_model = grid_search.best_estimator_
y_p = best_model.predict(X_uji)
score_LGBM = f1_score(y_uji, y_p, average="micro")
print(score_LGBM)

0; total time=   0.9s
Best Parameters: {'clf__learning_rate': 0.2, 'clf__max_bin': 120, 'clf__max_depth': 16, 'clf__n_estimators': 150, 'clf__num_iterations': 150}<br>
0.6129032258064516

# Random Forest

In [ ]:
pipa_RF3 = imbpipeline(steps=[
    ("preprocess", pre_3),
#     ("over", RandomOverSampler(random_state=42)),
    ("clf", RandomForestClassifier(random_state=42))
])

# Daftar parameter yang ingin dituning
param_grid = {
    'clf__n_estimators': [150],
    'clf__max_depth': [17],
    'clf__min_samples_split': [2],
    'clf__min_samples_leaf': [1],
    'clf__max_features': ['log2']
}

scorer = make_scorer(f1_score, average='micro')

# Inisialisasi GridSearchCV
grid_RF3 = GridSearchCV(estimator = pipa_RF2, param_grid = param_grid, cv = 5, verbose = 2, scoring = scorer)

# Melakukan pencarian parameter terbaik
grid_RF3.fit(X1, y1)

# Menampilkan parameter terbaik
print("Best Parameters:", grid_RF3.best_params_)

# Melakukan prediksi dengan model terbaik
best_model = grid_RF3.best_estimator_
y_p = best_model.predict(X_uji)
score_RF = f1_score(y_uji, y_p, average="micro")
print(score_RF)

Best Parameters: {'clf__max_depth': 19, 'clf__max_features': 'log2', 'clf__min_samples_split': 2, 'clf__n_estimators': 150}<br>
0.6709677419354839

In [ ]:
print(classification_report(y_uji, y_p))

> ### KNN

In [ ]:
pipa_KNN3 = imbpipeline(steps=[
    ("preprocess", pre_3),
#     ("over", RandomOverSampler(random_state=42)),
    ("clf", KNeighborsClassifier())
])

# Daftar parameter yang ingin dituning
param_grid = {
    'clf__n_neighbors': [5],
    'clf__weights': ['distance'],
    'clf__algorithm': ['auto'],
    'clf__p': [2]
}

scorer = make_scorer(f1_score, average='micro')

# Inisialisasi GridSearchCV
grid_KNN3 = GridSearchCV(estimator = pipa_KNN3, param_grid = param_grid, cv = 5, verbose = 2, scoring = scorer)

# Melakukan pencarian parameter terbaik
grid_KNN3.fit(X1, y1)

# Menampilkan parameter terbaik
print("Best Parameters:", grid_KNN3.best_params_)

# Melakukan prediksi dengan model terbaik
best_model = grid_KNN3.best_estimator_
y_p = best_model.predict(X_uji)
score_KNN = f1_score(y_uji, y_p, average="micro")
print(score_KNN)

Best Parameters: {'clf__algorithm': 'auto', 'clf__n_neighbors': 5, 'clf__p': 2, 'clf__weights': 'distance'}<br>
0.6516129032258065

> ### Logistic Regression

In [ ]:
pipa_LR3 = imbpipeline(steps=[
    ("preprocess", pre_3),
#     ("over", RandomOverSampler(random_state=42)),
    ("clf", LogisticRegression(random_state = 42))
])

# Daftar parameter yang ingin dituning
param_grid = {
    'clf__penalty': ['l1'],
    'clf__C': [1],
    'clf__solver': ['saga'],
    'clf__max_iter': [500]
}

scorer = make_scorer(f1_score, average='micro')

# Inisialisasi GridSearchCV
grid_LR3 = GridSearchCV(estimator = pipa_LR3, param_grid = param_grid, cv = 5, verbose = 2, scoring = scorer)

# Melakukan pencarian parameter terbaik
grid_LR3.fit(X1, y1)

# Menampilkan parameter terbaik
print("Best Parameters:", grid_LR3.best_params_)

# Melakukan prediksi dengan model terbaik
best_model = grid_LR3.best_estimator_
y_p = best_model.predict(X_uji)
score_LR = f1_score(y_uji, y_p, average="micro")
print(score_LR)

Best Parameters: {'clf__C': 1.0, 'clf__max_iter': 500, 'clf__penalty': 'l1', 'clf__solver': 'saga'}<br>
0.6451612903225806

> ### Extra Trees

In [ ]:
pipa_ETC3 = imbpipeline(steps=[
    ("preprocess", pre_3),
#     ("over", RandomOverSampler(random_state=42)),
    ("clf", ExtraTreesClassifier(random_state = 42))
])

# Daftar parameter yang ingin dituning
param_grid = {
    'clf__n_estimators': [200],
    'clf__max_depth': [None],
    'clf__min_samples_split': [2],
    'clf__min_samples_leaf': [1],
    'clf__max_features': ['log2']
}

scorer = make_scorer(f1_score, average='micro')

# Inisialisasi GridSearchCV
grid_ETC3 = GridSearchCV(estimator = pipa_ETC3, param_grid = param_grid, cv = 5, verbose = 2, scoring = scorer)

# Melakukan pencarian parameter terbaik
grid_ETC3.fit(X1, y1)

# Menampilkan parameter terbaik
print("Best Parameters:", grid_ETC3.best_params_)
# print(grid_ETC3.best_score_)

# Melakukan prediksi dengan model terbaik
best_model = grid_ETC3.best_estimator_
y_p = best_model.predict(X_uji)
score_ET = f1_score(y_uji, y_p, average="micro")
print(score_ET)

Best Parameters: {'clf__max_depth': None, 'clf__max_features': 'log2', 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2, 'clf__n_estimators': 200}<br>
0.6451612903225806

> ### LDA

In [ ]:
pipa_LDA3 = imbpipeline(steps=[
    ("preprocess", pre_3),
#     ("over", RandomOverSampler(random_state=42)),
    ("clf", LinearDiscriminantAnalysis())
])

# Daftar parameter yang ingin dituning
param_grid = {
    'clf__solver': ['svd'],
    'clf__shrinkage': [None],
    'clf__n_components': [None]
}

scorer = make_scorer(f1_score, average='micro')

# Inisialisasi GridSearchCV
grid_LDA3 = GridSearchCV(estimator = pipa_LDA3, param_grid = param_grid, cv = 5, verbose = 2, scoring = scorer)

# Melakukan pencarian parameter terbaik
grid_LDA3.fit(X1, y1)

# Menampilkan parameter terbaik
print("Best Parameters:", grid_LDA3.best_params_)
# print(grid_LDA3.best_score_)

# Melakukan prediksi dengan model terbaik
best_model = grid_LDA3.best_estimator_
y_p = best_model.predict(X_uji)
score_LDA = f1_score(y_uji, y_p, average="micro")
print(score_LDA)

Best Parameters: {'clf__n_components': None, 'clf__shrinkage': None, 'clf__solver': 'svd'}<br>
0.6129032258064516

> ### SVC

In [ ]:
#SVC 
pipa_SVC = imbpipeline(steps=[
    ("preprocess", pre_3),
    #("over", RandomOverSampler(random_state=42)),
    ("clf", SVC(random_state=56))
])

# Daftar parameter yang ingin dituning
param_grid = {
    'clf__kernel': ["rbf"],
    'clf__degree': [3],
    'clf__gamma': ["scale"],
    'clf__max_iter': [150]
}

# Inisialisasi GridSearchCV
grid_search = GridSearchCV(estimator=pipa_SVC, param_grid=param_grid, cv=5, verbose = 2, scoring=scorer)

# Melakukan pencarian parameter terbaik
grid_search.fit(X1, y1)

# Menampilkan parameter terbaik
print("Best Parameters:", grid_search.best_params_)
# Melakukan prediksi dengan model terbaik
best_model = grid_search.best_estimator_
y_p = best_model.predict(X_uji)
score_SVC = f1_score(y_uji, y_p, average="micro")
print(score_SVC)

Best Parameters: {'clf__degree': 3, 'clf__gamma': 'scale', 'clf__kernel': 'rbf', 'clf__max_iter': 150}<br>
0.6193548387096774

>### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier	
#bagging
pipa_Bagging = imbpipeline(steps=[
    ("preprocess", pre_3),
    #("over", RandomOverSampler(random_state=42)),
    ("clf", BaggingClassifier(random_state=56))
])

# Daftar parameter yang ingin dituning
param_grid = {
    'clf__n_estimators': [100],
    'clf__max_features' : [0.5],
    'clf__max_samples' : [1.0]
}

# Inisialisasi GridSearchCV
grid_search = GridSearchCV(estimator=pipa_Bagging, param_grid=param_grid, cv=5, verbose = 2, scoring=scorer)

# Melakukan pencarian parameter terbaik
grid_search.fit(X1, y1)

# Menampilkan parameter terbaik
print("Best Parameters:", grid_search.best_params_)
# Melakukan prediksi dengan model terbaik
best_model = grid_search.best_estimator_
y_p = best_model.predict(X_uji)
score_BC = f1_score(y_uji, y_p, average="micro")
print(score_BC)

Best Parameters: {'clf__max_features': 0.5, 'clf__max_samples': 1.0, 'clf__n_estimators': 100}<br>
0.6580645161290323

>### Ridge

In [ ]:
#Ridge
pipa_Ridge = imbpipeline(steps=[
    ("preprocess", pre_3),
    #("over", RandomOverSampler(random_state=42)),
    ("clf", RidgeClassifier())
])

# Daftar parameter yang ingin dituning
param_grid = {
    'clf__alpha': [1.2],
    'clf__fit_intercept': [True],
    'clf__solver': ["auto"],
    'clf__max_iter': [None]
}

# Inisialisasi GridSearchCV
grid_search = GridSearchCV(estimator=pipa_Ridge, param_grid=param_grid, cv=5, verbose = 2, scoring=scorer)

# Melakukan pencarian parameter terbaik
grid_search.fit(X1, y1)

# Menampilkan parameter terbaik
print("Best Parameters:", grid_search.best_params_)
# Melakukan prediksi dengan model terbaik
best_model = grid_search.best_estimator_
y_p = best_model.predict(X_uji)
score_RC = f1_score(y_uji, y_p, average="micro")
print(score_RC)

Best Parameters: {'clf__alpha': 1.2, 'clf__fit_intercept': True, 'clf__max_iter': None, 'clf__solver': 'auto'} <br>
0.5935483870967742

# MODEL TERBAIK

In [ ]:
score_model = pd.DataFrame({"Model": ["Random Forest", "KNN", "Logistic Regression", "Extra Trees", "LDA",
                                     "SVC", "Bagging Classifier", "Ridge Classifier"],
                           "F1 Score Micro": [score_RF, score_KNN, score_LR, score_ET,
                            score_LDA, score_SVC, score_BC, score_RC]
                           })
score_model

> ## Feature Importance

In [ ]:
importance = grid_RF3.best_estimator_.named_steps['clf'].feature_importances_

# Mendapatkan nama fitur
feature_names = X1.columns

# Mengurutkan fitur berdasarkan pentingnya
sorted_indices = np.argsort(importance)
sorted_importance = importance[sorted_indices]
sorted_feature_names = feature_names[sorted_indices]

# Membuat bar plot
plt.figure(figsize=(10, 6))
plt.bar(sorted_feature_names, sorted_importance)
plt.xticks(rotation='vertical')
plt.xlabel('Fitur')
plt.ylabel('Importance')
plt.show()